# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../files/final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2116 entries, 0 to 2115
Data columns (total 94 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   match_id                                2116 non-null   object 
 1   goals                                   2116 non-null   float64
 2   assists                                 2116 non-null   float64
 3   direct_goal_contributions               2116 non-null   float64
 4   non_penalty_goals                       2116 non-null   float64
 5   penalties_scored                        2116 non-null   float64
 6   penalties_attempted                     2116 non-null   float64
 7   yellow_cards                            2116 non-null   float64
 8   red_cards                               2116 non-null   float64
 9   expected_goals                          2116 non-null   float64
 10  non_penalty_expected_goals              2116 non-null   floa

In [4]:
combined.head()

,match_id,goals,assists,direct_goal_contributions,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,...,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_team_at_home_mean_goal_difference,home_team_overall_mean_goal_difference,away_team_at_away_mean_goal_difference,away_team_overall_mean_goal_difference,head_to_head_goal_difference
0,m-00381,0.393401,0.269112,0.662513,0.355397,0.038004,0.054601,0.556054,-0.031945,0.394375,...,9,2,2,0,0,2.4,1.6,-2.8,0.2,7
1,m-00382,0.134401,0.408515,0.542916,0.169614,-0.035212,-0.062705,0.464746,0.040671,0.203698,...,14,5,2,0,0,1.4,1.6,-0.6,1.2,1
2,m-00383,1.032333,0.289712,1.322045,0.976505,0.055828,0.080853,0.654894,-0.011413,0.903525,...,14,2,5,0,0,1.0,1.6,-1.4,0.2,8
3,m-00384,0.073795,0.225465,0.299260,0.152876,-0.079081,-0.080597,0.105934,-0.043515,-0.051271,...,15,0,2,0,0,2.0,1.6,0.0,0.6,1
4,m-00385,-0.308318,-0.074280,-0.382598,-0.265916,-0.042402,-0.029883,0.243744,-0.032462,-0.213221,...,12,4,1,0,0,1.8,1.6,1.0,0.0,3


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","total_passing_distance",
	"total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed",
	"medium_passes_attempted","long_passes_completed","long_passes_attempted","expected_assists","key_passes",
	"passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target",
	"average_shot_distance","shots_from_free_kicks","touches_in_defensive_penalty_area","touches_in_defensive_third",
	"touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches",
	"take_ons_attempted","take_ons_succeeded","carries","total_carrying_distance","progressive_carrying_distance",
	"carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received",
	"progressive_passes_received","tackles_won","defensive_third_tackles","middle_third_tackles","attacking_third_tackles",
	"dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances",
	"errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced",
	"penalties_allowed","penalties_saved","penalties_missed"
]
player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../files/feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = len(stats_columns)
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


2024-04-26 15:49:27.787169: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


34/34 [==============================] - 0s 401us/step - loss: 32.0763 - accuracy: 0.4332
Epoch 2/500
34/34 [==============================] - 0s 336us/step - loss: 21.6509 - accuracy: 0.4639
Epoch 3/500
34/34 [==============================] - 0s 366us/step - loss: 14.1393 - accuracy: 0.4740
Epoch 4/500
34/34 [==============================] - 0s 567us/step - loss: 9.9151 - accuracy: 0.4740
Epoch 5/500
34/34 [==============================] - 0s 515us/step - loss: 8.9028 - accuracy: 0.4740
Epoch 6/500
34/34 [==============================] - 0s 383us/step - loss: 8.6739 - accuracy: 0.4740
Epoch 7/500
34/34 [==============================] - 0s 332us/step - loss: 8.8995 - accuracy: 0.4740
Epoch 8/500
34/34 [==============================] - 0s 326us/step - loss: 8.6467 - accuracy: 0.4740
Epoch 9/500
34/34 [==============================] - 0s 313us/step - loss: 8.5337 - accuracy: 0.4740
Epoch 10/500
34/34 [==============================] - 0s 352us/step - loss: 8.5803 - accuracy: 0.474

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 312us/step
[ 1.2124596   1.7185048  11.282983   14.202717    3.9279435   5.129383
  4.5802526   5.707016   10.400647   10.449808    1.762317    1.6053094
  0.12943274  0.07501248]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 358us/step - loss: 28.4443 - accuracy: 0.3877
Epoch 2/500
34/34 [==============================] - 0s 337us/step - loss: 16.3734 - accuracy: 0.4681
Epoch 3/500
34/34 [==============================] - 0s 358us/step - loss: 11.9103 - accuracy: 0.4740
Epoch 4/500
34/34 [==============================] - 0s 335us/step - loss: 10.2658 - accuracy: 0.4746
Epoch 5/500
34/34 [==============================] - 0s 337us/step - loss: 9.3806 - accuracy: 0.4740
Epoch 6/500
34/34 [==============================] - 0s 334us/step - loss: 9.0001 - accuracy: 0.4740
Epoch 7/500
34/34 [==============================] - 0s 369us/step - loss: 8.8488 - accuracy: 0.4740
Epoch 8/500
34/34 [==============================] - 0s 361us/step - loss: 8.8323 - accuracy: 0.4746
Epoch 9/500
34/34 [==============================] - 0s 361us/step - loss: 8.8536 - accuracy: 0.4734
Epoch 10/500
34/34 [==============================] - 0s 367us/step - loss: 8.6836 - ac

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 369us/step
[ 0.7772789   1.7570287  11.785889   13.547287    3.6268342   4.9409003
  5.2710004   5.3752995  11.710075   10.995172    1.960809    1.7900858
  0.11458948  0.11596976]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


34/34 [==============================] - 0s 406us/step - loss: 31.3664 - accuracy: 0.3948
Epoch 2/500
34/34 [==============================] - 0s 369us/step - loss: 15.1423 - accuracy: 0.4740
Epoch 3/500
34/34 [==============================] - 0s 352us/step - loss: 12.1763 - accuracy: 0.4740
Epoch 4/500
34/34 [==============================] - 0s 337us/step - loss: 10.5479 - accuracy: 0.4740
Epoch 5/500
34/34 [==============================] - 0s 335us/step - loss: 9.6538 - accuracy: 0.4740
Epoch 6/500
34/34 [==============================] - 0s 330us/step - loss: 9.1906 - accuracy: 0.4740
Epoch 7/500
34/34 [==============================] - 0s 321us/step - loss: 8.9581 - accuracy: 0.4740
Epoch 8/500
34/34 [==============================] - 0s 352us/step - loss: 8.8021 - accuracy: 0.4740
Epoch 9/500
34/34 [==============================] - 0s 320us/step - loss: 8.7462 - accuracy: 0.4746
Epoch 10/500
34/34 [==============================] - 0s 359us/step - loss: 8.7787 - accuracy: 0.47

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 381us/step
[ 1.0977728   1.6483128  11.347124   12.520644    3.8303301   4.8437705
  5.4086847   5.0673423  10.804641   10.90645     1.9537805   1.7016736
  0.01374692  0.14879748]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(len(X_train.columns), activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


34/34 [==============================] - 0s 463us/step - loss: 27.7543 - accuracy: 0.3375
Epoch 2/500
34/34 [==============================] - 0s 365us/step - loss: 14.3499 - accuracy: 0.4752
Epoch 3/500
34/34 [==============================] - 0s 362us/step - loss: 11.7694 - accuracy: 0.4746
Epoch 4/500
34/34 [==============================] - 0s 342us/step - loss: 10.4762 - accuracy: 0.4740
Epoch 5/500
34/34 [==============================] - 0s 375us/step - loss: 9.7373 - accuracy: 0.4740
Epoch 6/500
34/34 [==============================] - 0s 306us/step - loss: 9.2541 - accuracy: 0.4740
Epoch 7/500
34/34 [==============================] - 0s 364us/step - loss: 9.0299 - accuracy: 0.4740
Epoch 8/500
34/34 [==============================] - 0s 378us/step - loss: 8.8849 - accuracy: 0.4746
Epoch 9/500
34/34 [==============================] - 0s 374us/step - loss: 8.8104 - accuracy: 0.4740
Epoch 10/500
34/34 [==============================] - 0s 381us/step - loss: 8.6834 - accuracy: 0.47

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 384us/step
[ 1.6675802   1.126408   14.469954   11.071317    5.0462947   3.4119895
  6.302296    4.872863   11.672959   10.508727    1.5564696   1.9868162
  0.37226063 -0.0790398 ]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
34/34 [==============================] - 0s 400us/step - loss: 26.7347 - accuracy: 0.3853
Epoch 2/500
34/34 [==============================] - 0s 414us/step - loss: 13.6705 - accuracy: 0.4716
Epoch 3/500
34/34 [==============================] - 0s 366us/step - loss: 11.3475 - accuracy: 0.4740
Epoch 4/500
34/34 [==============================] - 0s 362us/step - loss: 10.1118 - accuracy: 0.4740
Epoch 5/500
34/34 [==============================] - 0s 332us/step - loss: 9.5370 - accuracy: 0.4734
Epoch 6/500
34/34 [==============================] - 0s 346us/step - loss: 9.1581 - accuracy: 0.4758
Epoch 7/500
34/34 [==============================] - 0s 375us/step - loss: 8.9654 - accuracy: 0.4746
Epoch 8/500
34/34 [==============================] - 0s 376us/step - loss: 8.8923 - accuracy: 0.4752
Epoch 9/500
34/34 [==============================] - 0s 372us/step - loss: 8.8352 - accuracy: 0.4758
Epoch 10/500
34/34 [==============================] - 0s 377us/step - loss: 8.7630 - ac

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

14/14 [==============================] - 0s 350us/step
[1.07700253e+00 1.37302089e+00 7.00140858e+00 1.31727705e+01
 2.30232787e+00 4.05155325e+00 2.64847469e+00 5.34908867e+00
 1.13538284e+01 1.02288389e+01 1.67191160e+00 1.27275813e+00
 9.00942385e-02 1.05844997e-02]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1634,2,1,13,16,4,3,3,5,8,10,1,1,0,0
1479,2,4,16,15,6,9,3,3,13,19,6,3,0,0
25,2,1,6,23,2,6,4,8,14,11,4,2,0,0
1686,2,1,12,17,4,5,6,8,11,7,3,1,0,0
1454,1,3,9,15,3,7,3,8,8,15,0,2,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  penalties_scored  penalties_attempted  yellow_cards  \
618   0.282813  0.073807          0.012862             0.047048     -0.051462   
1581  1.264687  0.842364          0.022639             0.009442      0.984779   
1600  0.043288 -0.022940         -0.025762            -0.021131      0.497787   
812  -0.163778 -0.007934         -0.018938            -0.049900      0.391365   
2113 -0.150000 -0.039141         -0.012998            -0.012998     -0.092022   
...        ...       ...               ...                  ...           ...   
1542 -0.093617 -0.254741         -0.014639            -0.013673     -0.627743   
2062 -0.081567  0.127542         -0.037198            -0.056972      0.147570   
479  -0.660662 -0.268441         -0.115936            -0.129202     -0.484446   
561  -0.200871  0.054574          0.020231             0.000382     -0.386519   
1785 -0.501214 -0.169302         -0.016552            -0.023804     -0.299180   

      red_cards  expected_g

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../files/prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
# n=15
# print(X_train)
# pca = PCA(n_components = n, random_state=576)
# pca.fit(X)
# feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
# feature_to_pc_map.to_csv("../files/feature_to_15_pcs.csv")

# dump(pca, '../prediction_pca.bin')
# whole_X_pca = pca.transform(whole_x_train)

# pca = PCA(n_components = n, random_state=576)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)


# feature_to_pc_map

The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,progressive_carries,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,0.116731,0.130807,0.068462,0.070366,0.138851,0.070672,0.122732,0.123745,0.137735,0.138465,...,0.120774,0.115376,0.061499,0.062034,0.061952,0.054179,0.060231,0.058169,0.049960,0.036205
1,-0.097904,-0.088347,-0.059293,-0.063040,0.000947,0.013895,-0.092081,-0.091024,-0.084689,-0.080298,...,0.036963,0.160522,0.292189,0.295870,0.296043,0.277779,0.287178,0.275409,0.237882,0.187555
2,0.225268,0.119239,0.196718,0.201206,-0.100533,-0.071213,0.202350,0.189990,0.102673,0.072463,...,-0.214285,-0.055732,0.148153,0.146854,0.145492,0.135588,0.152631,0.147924,0.124193,0.092133
3,-0.165408,0.083206,-0.549854,-0.540602,-0.067493,-0.060127,-0.161756,-0.087649,0.081727,0.162701,...,-0.127337,-0.037406,0.032941,0.038853,0.040951,0.041342,0.042161,0.036456,0.028549,0.068472
4,0.138500,-0.126975,-0.142161,-0.150543,0.080946,0.367215,0.172220,0.216124,-0.126591,0.002635,...,0.171313,0.043778,-0.011955,-0.011159,-0.010366,-0.025213,0.001638,0.010327,-0.038101,0.005989
5,0.122177,0.069246,-0.167698,-0.160525,-0.052708,-0.656351,0.119503,0.158673,0.034022,0.001099,...,0.136159,0.029164,-0.025356,-0.027584,-0.027602,-0.031164,-0.067249,-0.039864,-0.098372,-0.151533
6,-0.038597,-0.114819,0.138349,0.140664,0.085118,-0.619954,-0.000610,-0.024317,-0.082234,0.014929,...,0.051123,0.030564,0.014677,0.023304,0.027813,-0.010817,0.035915,0.012260,-0.033513,0.301184
7,0.045653,0.069693,-0.077506,-0.070476,-0.011168,0.107785,0.041485,0.057609,0.050112,-0.052580,...,0.051462,0.057612,-0.105257,-0.067103,-0.048420,0.028871,-0.057043,-0.116285,-0.294706,0.861393
8,0.078044,0.026422,-0.236614,-0.211838,0.124874,-0.032302,0.105047,0.151435,0.041902,-0.185308,...,0.151517,0.046660,0.063285,0.053320,0.049456,-0.036935,0.074086,0.127042,-0.049368,-0.171963


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(INPUT_SIZE, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500


68/68 [==============================] - 0s 399us/step - loss: 28.3785 - accuracy: 0.4486
Epoch 2/500
68/68 [==============================] - 0s 341us/step - loss: 19.8542 - accuracy: 0.4699
Epoch 3/500
68/68 [==============================] - 0s 324us/step - loss: 18.8285 - accuracy: 0.4699
Epoch 4/500
68/68 [==============================] - 0s 318us/step - loss: 18.3038 - accuracy: 0.4699
Epoch 5/500
68/68 [==============================] - 0s 314us/step - loss: 18.1611 - accuracy: 0.4699
Epoch 6/500
68/68 [==============================] - 0s 316us/step - loss: 18.1385 - accuracy: 0.4699
Epoch 7/500
68/68 [==============================] - 0s 318us/step - loss: 18.0639 - accuracy: 0.4699
Epoch 8/500
68/68 [==============================] - 0s 315us/step - loss: 18.0271 - accuracy: 0.4699
Epoch 9/500
68/68 [==============================] - 0s 337us/step - loss: 17.9811 - accuracy: 0.4699
Epoch 10/500
68/68 [==============================] - 0s 330us/step - loss: 17.9368 - accuracy

In [33]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 493us/step - loss: 12.2229 - accuracy: 0.3703


[12.222949028015137, 0.37028300762176514]

#### Testing

In [34]:
y_hat = model.predict(X_test)

14/14 [==============================] - 0s 386us/step


In [35]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [36]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if home_shots_hat+away_shots_hat == home_shots+away_shots else 0
	total_shots_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots else 0
	total_shots_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if home_shots_on_target_hat+away_shots_on_target_hat == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if home_goals_hat == home_goals and away_goals_hat == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if home_fouls_hat + away_fouls_hat == home_fouls + away_fouls else 0
	total_fouls_over += 1 if home_fouls_hat + away_fouls_hat > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if home_fouls_hat + away_fouls_hat < home_fouls+ away_fouls else 0

	total_corners += 1 if home_corners_hat + away_corners_hat == home_corners + away_corners else 0
	total_corners_over += 1 if home_corners_hat + away_corners_hat > home_corners + away_corners else 0
	total_corners_under += 1 if home_corners_hat + away_corners_hat < home_corners + away_corners else 0

	goals_over += 1 if home_goals_hat + away_goals_hat > home_goals + away_goals else 0
	goals_under += 1 if home_goals_hat + away_goals_hat < home_goals + away_goals else 0

	all_under += 1 if home_shots_hat+away_shots_hat < home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat < home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat < home_corners + away_corners and home_corners_hat + away_corners_hat < home_corners + away_corners and home_goals_hat + away_goals_hat < home_goals + away_goals else 0
	all_over += 1 if home_shots_hat+away_shots_hat > home_shots+away_shots and home_shots_on_target_hat+away_shots_on_target_hat > home_shots_on_target+away_shots_on_target and (home_yellow_cards_hat+away_yellow_cards_hat)*10+(home_red_cards_hat+away_red_cards_hat)*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and home_corners_hat + away_corners_hat > home_corners + away_corners and home_goals_hat + away_goals_hat > home_goals + away_goals else 0

In [37]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.0
total_shots_over: 0.5943396226415094
total_shots_under: 0.4056603773584906


total_shots_on_target: 0.0
total_shots_on_target_over: 0.6297169811320755
total_shots_on_target_under: 0.37028301886792453


total_booking_points: 0.0
total_booking_points_over: 0.5212264150943396
total_booking_points_under: 0.47877358490566035


correct_score: 0.0


winner: 0.41745283018867924


total_corners: 0.0
total_corners_over: 0.5589622641509434
total_corners_under: 0.4410377358490566


total_fouls: 0.0
total_fouls_over: 0.5330188679245284
total_fouls_under: 0.4669811320754717


goals_over: 0.5990566037735849
goals_under: 0.4009433962264151


all_over: 0.11084905660377359
all_under: 0.04481132075471698


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [38]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(64, activation="relu", input_dim=64))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer="sgd",
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [39]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [40]:
# define a grid of the hyperparameter search space
hidden_layer_one = [30]
learn_rate = [1e-2]
dropout = [0.1]
batch_size = [64]
epochs = [500]
n_h_layers = [3]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [41]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [42]:
searcher = GridSearchCV(estimator=model, n_jobs=-2, param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)

searchResults = searcher.fit(X_train, y_train)

bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print(f"[INFO] best score is {bestScore} using {bestParams}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits


2024-04-26 15:50:24.223682: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-26 15:50:24.223682: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-26 15:50:24.223752: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 1/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=30, learn_rate=0.01, n_h_layers=3;, score=0.294 total time=   4.5s
[CV 2/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=30, learn_rate=0.01, n_h_layers=3;, score=0.267 total time=   4.5s
[CV 3/3] END batch_size=64, dropout=0.1, epochs=500, hidden_layer_one=30, learn_rate=0.01, n_h_layers=3;, score=0.289 total time=   4.6s
[INFO] best score is 0.2834420356677787 using {'batch_size': 64, 'dropout': 0.1, 'epochs': 500, 'hidden_layer_one': 30, 'learn_rate': 0.01, 'n_h_layers': 3}


In [43]:
tuned_model = searcher.best_params_
tuned_model

{'batch_size': 64,
 'dropout': 0.1,
 'epochs': 500,
 'hidden_layer_one': 30,
 'learn_rate': 0.01,
 'n_h_layers': 3}

In [44]:
hidden_layer_one = tuned_model["hidden_layer_one"]
learn_rate = tuned_model["learn_rate"]
dropout = tuned_model["dropout"]
batch_size = tuned_model["batch_size"]
epochs = tuned_model["epochs"]
n_h_layers = tuned_model["n_h_layers"]

model = get_mlp_model(hidden_layer_one=30, learn_rate=0.01, dropout=0.1, n_h_layers=3)
model.fit(X_train, y_train, epochs=500, batch_size=64, )

model.save("../files/stats_regression_model.h5")
# pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500


27/27 [==============================] - 0s 622us/step - loss: 36.1945 - accuracy: 0.1874
Epoch 2/500
27/27 [==============================] - 0s 458us/step - loss: 15.5418 - accuracy: 0.4267
Epoch 3/500
27/27 [==============================] - 0s 507us/step - loss: 13.1023 - accuracy: 0.4610
Epoch 4/500
27/27 [==============================] - 0s 377us/step - loss: 12.5760 - accuracy: 0.4622
Epoch 5/500
27/27 [==============================] - 0s 406us/step - loss: 11.8001 - accuracy: 0.4693
Epoch 6/500
27/27 [==============================] - 0s 368us/step - loss: 11.5871 - accuracy: 0.4687
Epoch 7/500
27/27 [==============================] - 0s 402us/step - loss: 11.5719 - accuracy: 0.4704
Epoch 8/500
27/27 [==============================] - 0s 410us/step - loss: 11.5527 - accuracy: 0.4693
Epoch 9/500
27/27 [==============================] - 0s 409us/step - loss: 11.3599 - accuracy: 0.4681
Epoch 10/500
27/27 [==============================] - 0s 418us/step - loss: 11.1547 - accuracy

In [45]:
print(len
	  (X_train))

1692


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?